# Script to detect faces using dlib.

### Importing libraries

In [1]:
import gc
import os
import sys
import math
import glob
import tqdm
import random
import numpy as np
from tqdm import tqdm
from time import sleep

In [2]:
import pandas as pd
import xml.etree.cElementTree as ET

In [3]:
import cv2
import dlib
from imutils import face_utils
from skimage.feature import hog
from skimage import data,exposure

### Defining paths

In [4]:
path_folder_all_frames = '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_all_in_one'

In [5]:
path_folder = '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise'

## Write an xml file, given the folder of images and path for detector(optional)

### Generating an xml file

In [45]:
def writeXMLForDetectedFaces(path_folder_of_folders_of_images, detector,predictor, output_file_name = 'output',isCNN=False):
    """
    writes XML after detecting faces given source folder of video folders containing image frames, a landmark predictor and a face detector
    """
    
    #Writing an XML
    root = ET.Element("dataset")
    name = ET.SubElement(root, "name").text = "Labelled faces"
    comment = ET.SubElement(root, "comment").text = "These are labelled images from Bagamoyo"
    images = ET.SubElement(root, "images")
    
    list_folder_images = glob.glob(path_folder_of_folders_of_images + '/*')
    print("Number of folders found in the folder are: ", len(list_folder_images))
    for vid_no,vid_folder in enumerate(list_folder_images):
        list_path_frames = glob.glob(vid_folder+'/*')
        print("Number of images found in the folder number {}, {} are: ".format(vid_no,vid_folder), len(list_path_frames))
        try:
            for path_frame in list_path_frames:
                #add image to images in XML
                image_node = ET.SubElement(images, "image", file=path_frame.split('/')[-1])
#                 print(path_frame)
#             try:
                image=cv2.imread(path_frame)
                gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
                    
                if(isCNN):
                    dets = detector(gray,1)
                    rects = dlib.rectangles()
                    rects.extend([d.rect for d in dets])
                else:
                    rects = detector(gray,1)
                for i,rect in enumerate(rects):                    
                    shape = predictor(gray, rect)
                    shape = face_utils.shape_to_np(shape)
                    (x, y, w, h) = face_utils.rect_to_bb(rect)
                    #add box to image in XML
#                     print(x,y,w,h)
                    box = ET.SubElement(image_node, "box", height=str(h), left=str(x), top=str(y), width=str(w))
                #write XML
            tree = ET.ElementTree(root)
            dest_name = output_file_name+str(vid_no)+'.xml'
            tree.write(dest_name)
            print("xml written to {}".format(dest_name))            
        except KeyboardInterrupt:
            break
        except:
            continue
    
        


In [46]:
def saveXMLForHOGDetector(path_folder_of_folders_of_images,output_file_name='output'):
    args={"shape_predictor":"shape_predictor_68_face_landmarks.dat"}
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(args["shape_predictor"])
    writeXMLForDetectedFaces(path_folder_of_folders_of_images=path_folder_of_folders_of_images, detector=detector,predictor=predictor,output_file_name = output_file_name)

In [47]:
def saveXMLForCNNDetector(path_folder_of_folders_of_images,output_file_name='output'):
    args={"shape_predictor":"shape_predictor_68_face_landmarks.dat"}
    detector = dlib.cnn_face_detection_model_v1('mmod_human_face_detector.dat')
    predictor = dlib.shape_predictor(args["shape_predictor"])
    writeXMLForDetectedFaces(path_folder_of_folders_of_images=path_folder_of_folders_of_images, detector=detector,predictor=predictor,output_file_name = output_file_name,isCNN=True)

In [ ]:
saveXMLForHOGDetector(path_folder,'hog_output')

In [13]:
saveXMLForCNNDetector(path_folder,'cnn_output')

('Number of folders found in the folder are: ', 524)
('Number of images found in the folder number 0, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_00-male-surprise_20180516_091529_219393302 (5-29-2018 10-29-50 AM) are: ', 104)
(-2L, 303L, 423L, 423L)
(1025L, 646L, 423L, 422L)
(1025L, 646L, 423L, 422L)
(-2L, 346L, 423L, 423L)
('Number of images found in the folder number 1, /media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_frames_folder_wise/VIDEO_00-male-surprise_20180516_091541_1754978908 (5-27-2018 5-58-50 AM) are: ', 107)
(1025L, 646L, 423L, 422L)
(-79L, 262L, 507L, 507L)
(1025L, 646L, 423L, 422L)
(-79L, 262L, 507L, 507L)
xml written to cnn_output.xml


In [12]:
cnn_face_detector = dlib.cnn_face_detection_model_v1('mmod_human_face_detector.dat')

In [ ]:
writeXMLForDetectedFaces()